<a href="https://colab.research.google.com/github/alfredamboka/DeepTech5Yr/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import math
import ee
#import geemap
#ee.Authenticate()
#ee.Initialize()




install and import geemap for convertion on js to py
Only applicable to Colab. In other cases or environment install normally and comment the cell below.

In [5]:
try:
    import geemap
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print('geemap not found, installing via pip in Google Colab...')
        !pip install geemap --quiet
        import geemap
    else:
        print('geemap not found, please install via conda in your environment')

geemap not found, installing via pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.3 MB/s eta 0:00:00


##Convert GEE JS - Python
Code to automatically convert GEE code from JavaScript to Python
- The cell below is used.
- Any js code provided will be converted to python.
- Write python code and js code in different cells

In [6]:
javascript_code = """
// Series of strings here!

//var forest = forest2, built_up = built_up2, agricultural = agricultural2, bareland = bareland2, water = water2, study_area = study_area2;


//*
//Performing supervised classification for LULC
//identify changes over time(time series for forest change)
//*/
//training point

var drawingTools = Map.drawingTools();
// Only allow drawing points.
drawingTools.setDrawModes(['polygon']);

// Get the layers list.
var layers = drawingTools.layers();

// Assuming the order is as follows
var forestLayer = layers.get(0);
var built_upLayer = layers.get(1);
var agriculturalLayer = layers.get(2);
var barelandLayer = layers.get(3);
var waterLayer = layers.get(4);

// shapefiles
var vihiga = ee.FeatureCollection('FAO/GAUL/2015/level2')
.filter(ee.Filter.eq('ADM2_NAME', 'Vihiga'))//.geometry();

//var focus_area = ee.FeatureCollection('projects/ee-alfredamboka/assets/kakamega');


var geometry = vihiga.geometry()
//var kk_forest = focus_area.clip(geometry)
//var study_areas = study_area.geometry()


var L7_collection = ee.ImageCollection('LANDSAT/LE07/C02/T2_L2')
//.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))
.filterDate('2000-01-01', '2020-01-01')
.filter(ee.Filter.bounds(geometry))
.sort('CLOUD_COVER')
.select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5']);


var L8_data = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2014-01-01', '2020-01-01')
    .filter(ee.Filter.bounds(geometry))
    .sort('CLOUD_COVER')
    .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']);


// Make a cloud-free composite. (for landsat 8)
/*var L8_dataset = ee.Algorithms.Landsat.simpleComposite({
  collection: L8_data,
  asFloat: true
});
*/

var newdataset = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR");

function maskL8sr(newdataset){
  var cloudShadowBitMask = 1<<3;
  var cloudBitMask = 1<<5;
  var qa = newdataset.select('pixel_qa');
  var mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
             .and(qa.bitwiseAnd(cloudBitMask).eq(0));
  return newdataset.updateMask(mask).divide(10000)
                //.select(["SR_B[0-9]*"])
                .copyProperties(newdataset,["system:time_start"]);
}


var steplist = ee.List.sequence(2014,2020)
var filter = steplist.map(function(year){
  var startDate = ee.Date.fromYMD(year,5,1);
  var endDate = ee.Date.fromYMD(year,9,15);
  var dta = newdataset.filterDate(startDate, endDate)
                              .map(maskL8sr)
                               .median()
                               .set('sytem:time_start',startDate);
return dta;
});

 var image_collection = ee.ImageCollection(filter);
 print(image_collection)


// Applies scaling factors.
function applyScaleFactors(image) {
  var opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  //var thermalBands = image.select('SR_B.*').multiply(0.00341802).add(149.0);
  return image.addBands(opticalBands, null, true)
              //.addBands(thermalBands, null, true);
}

var L8_dataset = L8_data.map(applyScaleFactors);
var L7_dataset = L7_collection.map(applyScaleFactors);

// median cluster image.

var L8_dataset = L8_dataset.median().clip(geometry)
var L7_dataset = L7_dataset.median().clip(geometry)

var visualization = {
  bands: ['SR_B4', 'SR_B3', 'SR_B2'],
  min: 0.0,
  max: 0.3,
};

var vis = {
  bands: ['SR_B3', 'SR_B2', 'SR_B1'],
  min: 0.0,
  max: 0.3,
};

/*sentinel image- to select training points/locations*/

function maskS2clouds(image) {
  var qa = image.select('QA60');

  // Bits 10 and 11 are clouds and cirrus, respectively.
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;

  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
      .and(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask).divide(10000);
}


var border_vis = vihiga.style({color: '#FF0000', width: 2, fillColor: 'ff475701',
lineType: 'solid'
});

//Merge training inputs.
var train_data = forest.merge(built_up).merge(agricultural).merge(bareland).merge(water)

// Overlay the point on the image to get training data.
var training = L8_dataset.sampleRegions({
  collection: train_data,
  properties: ['landcover'],
  scale: 10
});

// Train a classifier.
var classifier = ee.Classifier.smileRandomForest(50).train({
  features: training,
  classProperty: 'landcover',
  inputProperties: L8_dataset.bandNames()
});

// Classify the image.
var lulc = L8_dataset.classify(classifier);

/*calculating area under forest then
perform  time series for forest cover change*/
/* using the classified image for trainingg and identification of forest
change  */

var gfdata = ee.Image('UMD/hansen/global_forest_change_2021_v1_9').clip(geometry);
var CoverVisParam = {
  bands: ['treecover2000'],
  min: 0,
  max: 100,
  palette: ['black', 'green']
};

var forest_change = gfdata.select(['loss']);
var lossAreaImage = forest_change.multiply(ee.Image.pixelArea());
var cover_gain = gfdata.select(['gain']);
var gainarea = cover_gain.multiply(ee.Image.pixelArea());

var lossYear = gfdata.select(['lossyear']);
var gainyear = gfdata.select(['lossyear']);
var lossByYear = lossAreaImage.addBands(lossYear).reduceRegion({
  reducer: ee.Reducer.sum().group({
    groupField: 1
    }),
  geometry:geometry,
  scale: 30,
  maxPixels: 1e9
});

//

var gainByYear = lossAreaImage.addBands(gainyear).reduceRegion({
  reducer: ee.Reducer.sum().group({
    groupField: 1
    }),
  geometry:geometry,
  scale: 30,
  maxPixels: 1e9
});

var statsFormatted = ee.List(lossByYear.get('groups'))
  .map(function(el) {
    var d = ee.Dictionary(el);
    return [ee.Number(d.get('group')).format("20%02d"), d.get('sum')];
  });
var statsDictionary = ee.Dictionary(statsFormatted.flatten());
print(statsDictionary);

var plot = ui.Chart.array.values({
  array: statsDictionary.values(),
  axis: 0,
  xLabels: statsDictionary.keys()
}).setChartType('ColumnChart')
  .setOptions({
    title: 'Yeary Forest cover change',
    hAxis: {title: 'Year', format: '####'},
    vAxis: {title: 'Area (square meters)'},
    legend: { position: "none" },
    lineWidth: 1,
    pointSize: 3
  });
print(plot);


print(lossByYear);


Map.addLayer(gfdata, CoverVisParam, 'change');


Map.setCenter(34.88809292905271,0.1762018393544, 13);
//Map.centerObject(focus_area);
Map.addLayer(L8_dataset, visualization, 'L7');
Map.addLayer(lulc, {min: 0, max: 3, palette: ['red', '#024711', 'brown', 'grey','#77eb63']}, 'LULC');
//Map.addLayer(L7_dataset, vis, 'L7');
//Map.addLayer(dataset, vis_param, 'S2_A')
Map.addLayer(border_vis, null, 'Vihiga')


"""

In [7]:
lines = geemap.js_snippet_to_py(
    javascript_code, add_new_cell=False,
    import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())

import ee
import geemap
Map = geemap.Map()
import geemap

Map = geemap.Map()

# Series of strings here!

#forest = forest2, built_up = built_up2, agricultural = agricultural2, bareland = bareland2, water = water2, study_area = study_area2

#*
#Performing supervised classification for LULC
#identify changes over time(time series for forest change)
##
#training point

drawingTools = Map.drawingTools()
# Only allow drawing points.
drawingTools.setDrawModes(['polygon'])

# Get the layers list.
layers = drawingTools.layers()

# Assuming the order is as follows
forestLayer = layers.get(0)
built_upLayer = layers.get(1)
agriculturalLayer = layers.get(2)
barelandLayer = layers.get(3)
waterLayer = layers.get(4)

# shapefiles
vihiga = ee.FeatureCollection('FAO/GAUL/2015/level2') \
.filter(ee.Filter.eq('ADM2_NAME', 'Vihiga'))

#focus_area = ee.FeatureCollection('projects/ee-alfredamboka/assets/kakamega')

geometry = vihiga.geometry()
#kk_forest = focus_area.clip(geometry)
#study_areas = study_area

Testing Cell Bellow toconfirm if it is working.

In [8]:
#More than python!

## Batch Export
- Create a function that will enhance batch export of any output required.